# Scrape Book Titles and Prices

In [4]:
import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/"
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup=BeautifulSoup(html, 'html.parser')

books = soup.find_all('article', class_='product_pod')

books_dict = {}

for book in books:
    title = book.h3.a['title']
    price_text = book.find('p', class_='price_color').text
    cleaned = ''.join(char for char in price_text if char.isdigit() or char == '.')
    price = float(cleaned)
    books_dict[title] = price

with open('books.txt', 'w', encoding='utf-8') as file:
    for title, price in books_dict.items():
        file.write(f"{price}, {title}\n")


# Parse Country Names and Capitals

In [15]:
import xml.etree.ElementTree as ET

url = "https://api.worldbank.org/v2/country?per_page=50"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()

    root = ET.fromstring(response.content)
    
    namespace = {'wb': 'http://www.worldbank.org'}
    countries_dict = {}

    for country in root.findall('.//wb:country', namespace):
        name_ele = country.find('wb:name', namespace)
        name = name_ele.text if name_ele is not None and name_ele.text else None
        capital_ele = country.find('wb:capitalCity', namespace)
        capital = capital_ele.text if capital_ele is not None and capital_ele.text and capital_ele.text.strip() else "N/A"

        if name:
            countries_dict[name] = capital

    with open('countries.txt', 'w', encoding='utf-8') as file:
        for name, capital in countries_dict.items():
            file.write(f"Country: {name}, Capital: {capital}\n")

except requests.RequestException as e:
    print(f"Error fetching data: {e}")